### Global Queue Retrieval

In [ ]:
import sys
sys.path.append(r'D://ASY//Project//Week_2')

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from configs.default import INDEX_FILE, DEVICE_DEFAULT
from build_index import build_annoy
from retriever import Retriever
from utils_bbox import draw_bbox

if not INDEX_FILE.exists():
    build_annoy(max_items=None)     
ret = Retriever(device=DEVICE_DEFAULT)

query = "a butterfly on a yellow flower"
results = ret.query(query, k=5)

for rnk, r in enumerate(results,1):
    img = Image.open(r["image_path"]).convert("RGB")
    draw_bbox(img, r["bbox"])
    plt.figure(figsize=(4,4))
    plt.title(f"rank {rnk} | score {r['score']:.3f}")
    plt.imshow(img); plt.axis("off"); plt.show()
    print(r["caption"],"\n")


### Image-Local Retrieval

In [ ]:
import random, json
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from configs.default import INDEX_FILE, DEVICE_DEFAULT, JSONL_TRAIN, IMAGES_DIR
from build_index import build_annoy
from retriever import Retriever
from utils_bbox import draw_traces, bbox_iou, traces_to_bbox

if not INDEX_FILE.exists():
    build_annoy(max_items=None)
ret = Retriever(device=DEVICE_DEFAULT)

# Random Aampling
with open(JSONL_TRAIN, encoding="utf-8") as f:
    lines = [json.loads(line) for line in f.readlines()]

samples = []
for entry in random.sample(lines, 10):
    img_id = entry["image_id"]
    caption = entry["caption"]
    img_path = None
    for ext in (".jpg", ".jpeg", ".png"):
        p = IMAGES_DIR / f"{img_id}{ext}"
        if p.exists():
            img_path = p
            break
    if not img_path:
        continue

    # Collect all trajectory points
    traces = []
    for subtrace in entry.get("traces", []):
        if isinstance(subtrace, list):
            traces.extend(subtrace)
        elif isinstance(subtrace, dict):
            traces.append(subtrace)

    img_w, img_h = Image.open(img_path).size
    bbox = traces_to_bbox(traces, img_w, img_h)

    samples.append({
        "caption": caption,
        "image_path": str(img_path),
        "traces": traces,
        "bbox": bbox       
    })

# visualization Top-5
for rnk, item in enumerate(samples, 1):
    print(f"=== Query {rnk} ===")
    print(f"Caption: {item['caption']}\n")
    query_caption = item["caption"]
    if len(query_caption) > 77:
        query_caption = query_caption[:77] 
    results = ret.query(query_caption, k=5)


    img = Image.open(item["image_path"]).convert("RGB")
    draw = ImageDraw.Draw(img)

    # Green
    radius = max(3, int(0.002 * max(img.size)))
    draw_traces(img, item["traces"], color="green", radius=radius)

    # Red
    width = max(3, int(0.003 * max(img.size)))
    for idx, res in enumerate(results, 1):
        draw.rectangle(res["bbox"], outline="red", width=width)
        x, y = res["bbox"][0], res["bbox"][1]
        draw.text((x, y), f"{idx}", fill="red")


    plt.figure(figsize=(8,8))
    plt.title(f"Query {rnk} — Top-5 Retrieved")
    plt.imshow(img)
    plt.axis("off")
    plt.show()



In [ ]:
from evaluate import evaluate_metrics
metrics = evaluate_metrics(ret, samples, k_list=(1,5), iou_thr=0.5)
print("\nEvaluation Results:")
for k,v in metrics.items():
    print(f"{k}: {v:.4f}")
